## Court Caseload Extract

In [8]:
!pip install pydbtools
!pip install numpy==1.24.3 --user --force-reinstall
!pip install "pybind11>=2.12" -force-reinstall

  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.1 requires numpy<1.23.0,>=1.16.5, but you have numpy 2.0.2 which is incompatible.
numba 0.54.0 requires numpy<1.21,>=1.17, but you have numpy 2.0.2 which is incompatible.
  Using cached numpy-1.24.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.1 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.3 which is incompatible.
numba 0.54.0 requires numpy<1.21,>=1

In [2]:
import pydbtools as pydb
import pandas as pd

## Quarterly Figures

In [3]:
# Query to get the dataset from athena
q_query = "Select * from courts_caseload_rpt_dev_dbt.rpt_crown_court_caseload_quarterly_summary"
dfq = pydb.read_sql_query(q_query)

#Get the first two rows of the dataset to explore
dfq.head(2)

,id,year,quarter,all_cases_receipts,all_cases_disposals,all_cases_opens,total_all_trial_receipts,total_all_trial_disposals,total_all_trial_opens,triable_either_way_receipts,...,indictable_only_opens,committed_for_sentence_receipts,committed_for_sentence_disposals,committed_for_sentence_opens,appeals_receipts,appeals_disposals,appeals_opens,unknown_receipts,unknown_disposals,unknown_opens
0,2f8d1c1e93f31d0765194a95718f891f,2025,Q4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,755b7c10d16531b8a315aad797aa801c,2025,Q3,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Rename Columns To Conform with Output specification 

Get Column Names from dbt config

In [4]:
!pip install pyyaml
import yaml

In [5]:
#Import DBT config file

file_path = r'courts_caseload_rpt__rpt_crown_court_caseload_properties.yml'
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)
print(data)

{'version': 2, 'models': [{'name': 'courts_caseload_rpt__rpt_crown_court_caseload_quarterly_summary', 'description': 'This model represents the staging component of the reporting layer, utilizing data from the OneCrown dimensional model', 'columns': [{'name': 'id', 'description': 'Unique Identifier for each row', 'tests': ['unique', 'not_null']}, {'name': 'year', 'description': 'Year', 'tests': ['not_null']}, {'name': 'quarter', 'description': 'Quarter', 'tests': ['not_null']}, {'name': 'all_cases_receipts', 'description': 'All cases:receipts'}, {'name': 'all_cases_disposals', 'description': 'All cases:disposals'}, {'name': 'all_cases_opens', 'description': 'All cases:open'}, {'name': 'total_all_trial_receipts', 'description': 'All trials:receipts'}, {'name': 'total_all_trial_disposals', 'description': 'All trials:disposals'}, {'name': 'total_all_trial_opens', 'description': 'All trials:open'}, {'name': 'triable_either_way_receipts', 'description': 'Triable-either-way trials:receipts'}

In [6]:
#Get Output Column Names from .yml description tags

updated_column_name = [] 
for model in data['models']:
    if 'description' in model:
        updated_column_name.append(model['description'])
    if 'columns' in model:
        for column in model['columns']:
            if 'description' in column:
                updated_column_name.append(column['description'])
del updated_column_name[:2]
print(updated_column_name) 

['Year', 'Quarter', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:open']


In [7]:
#Counts to check column (items) in list

count = len(updated_column_name)
print(count)

23


In [8]:
#Map Dataframe columns to updated columns from list

dfq = dfq.drop(columns='id')
dfq.columns = updated_column_name
column_list = dfq.columns.to_list()
print(column_list)

['Year', 'Quarter', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:open']


# Build GPTables for Quarterly figures

In [9]:
!pip install gptables

import gptables as gpt

  Using cached gptables-1.2.0-py3-none-any.whl (72 kB)
  Using cached XlsxWriter-3.2.2-py3-none-any.whl (165 kB)


In [11]:
# Define Metadata for the quarterly tables 

metadata = {
    "title": "Table C1: Receipts, disposals and open criminal cases in the Crown Court in England and Wales, annually 2016 - 2023, quarterly Q1 2016 - Q3 2024 [note 13][note 14][note 15][note 16][note 118] (“One Crown”)",
    "source": "Source: XHIBIT system and Common Platform, HMCTS",
}

#Create a GPTable object

q_table = gpt.GPTable(
    table=dfq,
    index_columns={1:0},
    title=metadata["title"],
    table_name="Crown_Court_Cases",
   ## subtitle=metadata["subtitle"],
    source=metadata["source"],
)

q_data = q_table.table

#Convert GPTable to Dataframe 
dfql = pd.DataFrame(q_data)



# Yearly Figures

In [12]:
# Query to get the dataset from athena

y_query = "Select * from courts_caseload_rpt_dev_dbt.rpt_crown_court_caseload_yearly_summary"
dfy = pydb.read_sql_query(y_query)

#Get the first two rows of the dataset to explore

dfy.head(2)

,id,year,all_cases_receipts,all_cases_disposals,all_cases_opens,total_all_trial_receipts,total_all_trial_disposals,total_all_trial_opens,triable_either_way_receipts,triable_either_way_disposals,...,indictable_only_opens,committed_for_sentence_receipts,committed_for_sentence_disposals,committed_for_sentence_opens,appeals_receipts,appeals_disposals,appeals_opens,unknown_receipts,unknown_disposals,unknown_opens
0,312351bff07989769097660a56395065,2025,14928,14455,75261,<NA>,<NA>,<NA>,6241,5594,...,21871,5202,5301,11599,571,754,2589,2,3,6
1,07811dc6c422334ce36a09ff5cd6fe71,2024,121571,113936,174941,<NA>,<NA>,<NA>,49267,43936,...,42049,43399,42505,46094,5419,5627,7586,23,20,23


# Rename To Conform with Output specification

In [13]:
#Import DBT config file

file_path_y = r'courts_caseload_rpt__rpt_crown_court_caseload_Y_properties.yml'
with open(file_path_y, 'r') as file:
    data_y = yaml.safe_load(file)
print(data_y)

{'version': 2, 'models': [{'name': 'courts_caseload_rpt__rpt_crown_court_caseload_yearly_summary', 'description': 'This model represents the base component of the reporting layer, utilizing data from the OneCrown dimensional model to create a yearly summary of the caseload report', 'columns': [{'name': 'id', 'description': 'Unique Identifier for each row', 'tests': ['unique', 'not_null']}, {'name': 'year', 'description': 'Year', 'tests': ['not_null']}, {'name': 'all_cases_receipts', 'description': 'All cases:receipts'}, {'name': 'all_cases_disposals', 'description': 'All cases:disposals'}, {'name': 'all_cases_opens', 'description': 'All cases:open'}, {'name': 'total_all_trial_receipts', 'description': 'All trials:receipts'}, {'name': 'total_all_trial_disposals', 'description': 'All trials:disposals'}, {'name': 'total_all_trial_opens', 'description': 'All trials:open'}, {'name': 'triable_either_way_receipts', 'description': 'Triable-either-way trials:receipts'}, {'name': 'triable_either

In [14]:
#Get Output Column Names from .yml description tags

updated_column_name_y = [] 
for model in data_y['models']:
    if 'description' in model:
        updated_column_name_y.append(model['description'])
    if 'columns' in model:
        for column in model['columns']:
            if 'description' in column:
                updated_column_name_y.append(column['description'])
del updated_column_name_y[:2]
print(updated_column_name_y) 

['Year', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:open']


In [15]:
#Counts to check column (items) in list

count = len(updated_column_name_y)
print(count)

22


In [16]:
#Map Dataframe columns to updated columns from list

dfy = dfy.drop(columns='id')
dfy.columns = updated_column_name_y
column_list_y = dfy.columns.to_list()
print(column_list_y)

['Year', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:open']


# Build GPtable for yearly figures

In [49]:
# Define Metadata for the quarterly tables 

metadata = {
    "title": "Table C1: Receipts, disposals and open criminal cases in the Crown Court in England and Wales, annually 2016 - 2023, quarterly Q1 2016 - Q3 2024 [note 13][note 14][note 15][note 16][note 118] (“One Crown”)",
    "subtitle1": "This worksheet contains one table.",
    "subtitle2": "This table contains notes, which can be found in the Notes worksheet.",
    "source": "Source: XHIBIT system and Common Platform, HMCTS",
}

#Create a GPTable object

y_table = gpt.GPTable(
    table=dfy,
    index_columns={1:0},
    title=metadata["title"],
    table_name="Crown_Court_Cases",
   ## subtitle=metadata["subtitle"],
    source=metadata["source"],
)
y_data = y_table.table

#Convert GPTable to Dataframe
dfyl = pd.DataFrame(y_data)

# Export Quarterly and yearly Figures to Excel Document

In [50]:
!pip install openpyxl
!pip install XlsxWriter

import openpyxl
import xlsxwriter
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment

In [51]:
# Save the table as an Excel File 

output_filename = "GPTable_Output.xlsx"
with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    dfyl.to_excel(writer, sheet_name="Yearly_Crown_Court_Cases", startrow=5, index=False)
    dfql.to_excel(writer, sheet_name="Quarterly_Crown_Court_Cases", startrow=5, index=False)

#Edit Yearly Figures WorkBook

#Load Workbook
wb = load_workbook(output_filename)

def insert_metadata(sheet_name):
    ws = wb[sheet_name]
    ws["A1"] = metadata["title"]  # Title in first row
    ws["A2"] = metadata["subtitle1"]  # subtitle1 in second row
    ws["A3"] = metadata["subtitle2"]  # subtitle2 in third row
    ws["A4"] = metadata["source"]  # Source in fourth row
    
    ws["A1"].font = Font(bold=True)  # Make title bold
    
    # Apply wrap text to row 4 (column headers)
    for col in range(1, ws.max_column + 1):  # Loop through all columns
        ws.cell(row=6, column=col).alignment = Alignment(wrap_text=True)

# Apply metadata to both sheets
insert_metadata("Yearly_Crown_Court_Cases")
insert_metadata("Quarterly_Crown_Court_Cases")

# Save the updated file
wb.save(output_filename)


Download Link 

In [52]:
from IPython.display import HTML

def create_download_link(filename):
    return HTML(f'<a href="{filename}" download>Click here to download {filename}</a>')

create_download_link(output_filename)

Copy Workbook To S3 Location

In [53]:
!pip install boto3

In [56]:
import boto3

In [58]:
# Set up AWS session with credentials
#session = boto3.Session(
#    aws_access_key_id="",
#    aws_secret_access_key="",
#    region_name=""
#)

# Create an S3 client using the session
s3_client = boto3.client("s3")

# Prepare the file and the bucket name
bucket_name = "alpha-fotest"  # Enclosed in quotes
output_filename = "GPTable_Output.xlsx"  # The file you're uploading
data = open(output_filename, "rb")  # Open the file in binary mode

response = s3_client.put_object(Bucket=bucket_name , Body=data, Key=output_filename)
print(f"AWS response code for uploading file is {(response['ResponseMetadata']['HTTPStatusCode'])}")


AWS response code for uploading file is 200
